In [17]:
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [18]:
!kaggle competitions download -c 2021-ai-w4-p1

2021-ai-w4-p1.zip: Skipping, found more recently modified local copy (use --force to force download)


In [19]:
!unzip 2021-ai-w4-p1.zip

Archive:  2021-ai-w4-p1.zip
replace submit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [20]:
import pandas as pd
import numpy as np

train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
submit=pd.read_csv("submit.csv")

In [21]:
x=train.drop(["Diabetes",'Pregnancies', 'SkinThickness'],axis=1)
y=train["Diabetes"]
test=test.drop(['Pregnancies', 'SkinThickness'],axis=1)

In [22]:
 from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler().fit(x)
x_std=transformer.transform(x)
test_std=transformer.transform(test)

In [23]:
import torch

seed=777

torch.manual_seed(seed)
device='cuda' if torch.cuda.is_available() else 'cpu'

if device=='cuda':
  torch.cuda.manual_seed_all(seed)

In [24]:
x=np.array(x_std)
y=np.array(y)
test=np.array(test_std)

x=torch.FloatTensor(x)
y=torch.LongTensor(y)
test=torch.FloatTensor(test)

In [25]:
linear1=torch.nn.Linear(6,32)
linear2=torch.nn.Linear(32,32)
linear3=torch.nn.Linear(32,16)
linear4=torch.nn.Linear(16,2)

relu=torch.nn.ReLU()

dropout=torch.nn.Dropout(p=0.1)

In [26]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)

Parameter containing:
tensor([[-0.4760,  0.4223, -0.5434,  0.5320, -0.4806,  0.0748, -0.5100,  0.5033,
         -0.0476,  0.0642, -0.5354,  0.5075,  0.0394, -0.4953, -0.0813, -0.3254],
        [-0.3587,  0.1029,  0.4550, -0.5662, -0.3328,  0.0869,  0.0989, -0.2617,
          0.1658, -0.2632,  0.0279,  0.1195,  0.1574, -0.5574,  0.4846, -0.0268]],
       requires_grad=True)

In [27]:
model=torch.nn.Sequential(linear1,relu,dropout,linear2,relu,dropout,linear3,relu,dropout,linear4).to(device)
optimizer = torch.optim.Adam(params= model.parameters(), lr=0.001)
loss=torch.nn.CrossEntropyLoss().to(device)

In [28]:
model.train()
for stop in range(1000):
  optimizer.zero_grad()
  h=model(x.to(device))
  cost=loss(h,y.to(device))
  cost.backward()
  optimizer.step()
  if stop%1000==0:
    print(stop,cost.item())

0 0.7780413627624512


In [29]:
pred=list()
with torch.no_grad():
  model.eval()
  test_y=model(test.to(device))
  a,b=torch.max(test_y,dim=1)
  pred.extend(b.tolist())

In [30]:
submit["Diabetes"]=pred
submit.to_csv("submit.csv",index=False)

In [31]:
submit

,ID,Diabetes
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
226,226,0
227,227,0
228,228,0
229,229,0


In [32]:
!kaggle competitions submit -c 2021-ai-w4-p1 -f submit.csv -m "Message"

100% 1.26k/1.26k [00:03<00:00, 424B/s]
Successfully submitted to 당뇨병 예측